In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../data/sales.csv')
df.dropna(subset=['price'], inplace=True)

In [3]:
df.head()

,cost,price,weight,purchase_date,product_type,product_level,maker,ingredient,height,width,depth
0,$333k,"$300,492",3 Ton 90 Kg,Dec 19 2008,"Q,B",advanced,M14122,"IN732052,IN732053",2.76 meters,97 cm,26 cm
1,NaN,"$430,570",3 Ton 30 Kg,Sep 10 1997,"J,D",basic,NaN,"IN732054,IN732055,IN732056,IN732057,IN732058",2.67 meters,98 cm,26 cm
2,$270k,"$213,070",3 Ton 40 Kg,Sep 05 2001,"J,D",basic,NaN,"IN732054,IN732059,IN732060",3.0 meters,93 cm,24 cm
3,NaN,"$229,174",3 Ton 50 Kg,Dec 23 2016,U,advanced,M14123,"IN732061,IN732062,IN732063",2.5 meters,102 cm,27 cm
4,$97k,"$122,659",2 Ton 970 Kg,Jan 12 2000,"D,R",advanced,NaN,"IN732064,IN732065,IN732066",2.47 meters,101 cm,26 cm


In [4]:
df['year'] = pd.to_datetime(df.purchase_date).dt.year
train_raw = df[df.year < 2015]
test_raw = df[df.year >= 2015]

# 1. Model building

<font color='red'>Assignment:</font> Get **luxury** (**price** higher than 500k dollars) as targets for training and testing and assign them to variables **y_train** and **y_test**, respectively.

In [5]:
y_train = train_raw['price'].map(lambda x: float(x.strip('$').replace(',',''))).map(lambda x: 1 if x > 500000 else 0)
y_test = test_raw['price'].map(lambda x: float(x.strip('$').replace(',',''))).map(lambda x: 1 if x > 500000 else 0)

<font color='red'>Assignment:</font> Build a transformation class to extract, preprocess, and transform **cost**; wrap up this class with **MinMaxScaler** and **LogisticRegression** to build a classification pipeline.

In [13]:
class cost_transform(object):
    
    def fit(self, X_train, y=None):
        self.mean_price = X_train['cost'].map(lambda x: x if type(x) == float else float(x.strip('$').strip('k'))*1000).mean()
        
    def transform(self, X_train, y=None):
        df = pd.DataFrame()
        df['cost'] = X_train['cost'].map(lambda x: x if type(x) == float else float(x.strip('$').strip('k'))*1000)
        return df.fillna(value = self.mean_price)

    def fit_transform(self, X_train, y=None):
        self.fit(X_train)
        return self.transform(X_train)
    

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import LogisticRegression

steps = [('ctf', cost_transform()),
             ('rescale', MinMaxScaler()),
             ('lr', LogisticRegression(random_state=2023))]

model = Pipeline(steps)
model.fit(train_raw, y_train)

Pipeline(steps=[('ctf', <__main__.cost_transform object at 0x000002144CF571F0>),
                ('rescale', MinMaxScaler()),
                ('lr', LogisticRegression(random_state=2023))])

<font color='red'>Question:</font> What is logistic regression? Is logistic regression a regression algorithm?

In [ ]:
#Logistic regression estimates the probability of the ocurring of an event based on the values of independent variables.
# Logistic regression is a classification algorithm. 

<font color='red'>Question:</font> Is logistic regression a linear algorithm? What is the relationship between logistic regression and linear regression?

In [ ]:
# Logistic regression is not linear itself, but during the logistic regression a logit function transforms the odd of the event 
# into a log odds, after that the fitting of log odds by independent variables is linear, similar to linear regression.

<font color='red'>Question:</font> What is sigmoid function? What is it for?

In [ ]:
# Sigmoid function is a math function mapping any real values into prabability between 0 and 1. In logistic regression, sigmoid 
#  function maps the linear combinations of independent variables into the propability of the event.

# 2. Predicting

## 2.1 Prediction with default settings

<font color='red'>Assignment:</font> Use the **predict** function of the model to make predictions for the training set and test set, and assign the outputs to **y_train_pred** and **y_test_pred**, respectively.

In [14]:
y_train_pred = model.predict(train_raw)
y_test_pred = model.predict(test_raw)

<font color='red'>Question:</font> What are the shapes of **y_train_pred** and **y_test_pred**? What do the values in **y_train_pred** and **y_test_pred** mean?

In [20]:
# the shape of y_train_pred and y_test_pred is a array of the same size to y_train and y_test, the value is 0 or 1, representing
# whether the pridicted price is >500K or not.

<font color='red'>Assignment:</font> Calculate the training and testing accuracy scores.

In [22]:
train_accuracy = sum(y_train_pred == y_train.values) / len(y_train)
test_accuracy = sum(y_test_pred == y_test.values) / len(y_test)

print('train accuracy: {0:.2e}'.format(train_accuracy))
print('test accuracy: {0:.2e}'.format(test_accuracy))

train accuracy: 9.40e-01
test accuracy: 9.07e-01


<font color='red'>Assignment:</font> Calculate the precision, recall, and f1 scores. You can use classification report.

In [23]:
from sklearn.metrics import classification_report

target_names = ['Non luxury', 'Luxury']
print('Train report')
print(classification_report(y_train, y_train_pred, target_names=target_names))

Train report
              precision    recall  f1-score   support

  Non luxury       0.94      0.99      0.97      2573
      Luxury       0.70      0.17      0.27       184

    accuracy                           0.94      2757
   macro avg       0.82      0.58      0.62      2757
weighted avg       0.93      0.94      0.92      2757



In [24]:
print('Test report')
print(classification_report(y_test, y_test_pred, target_names=target_names))

Test report
              precision    recall  f1-score   support

  Non luxury       0.91      0.99      0.95       383
      Luxury       0.71      0.22      0.33        46

    accuracy                           0.91       429
   macro avg       0.81      0.60      0.64       429
weighted avg       0.89      0.91      0.88       429



<font color='red'>Assignment:</font> Get the confusion matrix.

In [25]:
from sklearn.metrics import confusion_matrix

print('Train confusion matrix')
confusion_matrix(y_train, y_train_pred)

Train confusion matrix


array([[2560,   13],
       [ 153,   31]], dtype=int64)

In [26]:
print('Test confusion matrix')
confusion_matrix(y_test, y_test_pred)

Test confusion matrix


array([[379,   4],
       [ 36,  10]], dtype=int64)

<font color='red'>Question:</font> Is this model better than the all-positive and all-negative models?

In [ ]:
# I pull the testing scores from the all three models:
# all-neg: precision = 0.00, recall = 0.00, f1 = 0.00
# all-pos: precision = 0.11, recall = 1.00, f1 = 0.19
# log-reg: precision = 0.71, recall = 0.22, f1 = 0.33

# All positive models has best recall, logistic regression has best precision and f1, so it is better than the other two models.

## 2.2 Prediction with balanced class weights

<font color='red'>Assignment:</font> Set class_weight in Logistic Regression and retrain the model.

In [27]:
steps = [('ctf', cost_transform()),
             ('rescale', MinMaxScaler()),
             ('lr', LogisticRegression(class_weight='balanced', random_state=2023))]

model = Pipeline(steps)
model.fit(train_raw, y_train)

Pipeline(steps=[('ctf', <__main__.cost_transform object at 0x000002144A6368B0>),
                ('rescale', MinMaxScaler()),
                ('lr',
                 LogisticRegression(class_weight='balanced',
                                    random_state=2023))])

<font color='red'>Assignment:</font> Re-calcualte all the above metrics and confusion matrix.

In [28]:
y_train_pred = model.predict(train_raw)
y_test_pred = model.predict(test_raw)

In [29]:
train_accuracy = sum(y_train_pred == y_train.values) / len(y_train)
test_accuracy = sum(y_test_pred == y_test.values) / len(y_test)

print('train accuracy: {0:.2e}'.format(train_accuracy))
print('test accuracy: {0:.2e}'.format(test_accuracy))

train accuracy: 8.58e-01
test accuracy: 8.72e-01


In [30]:
target_names = ['Non luxury', 'Luxury']
print('Train report')
print(classification_report(y_train, y_train_pred, target_names=target_names))

Train report
              precision    recall  f1-score   support

  Non luxury       0.98      0.87      0.92      2573
      Luxury       0.28      0.71      0.40       184

    accuracy                           0.86      2757
   macro avg       0.63      0.79      0.66      2757
weighted avg       0.93      0.86      0.88      2757



In [31]:
print('Test report')
print(classification_report(y_test, y_test_pred, target_names=target_names))

Test report
              precision    recall  f1-score   support

  Non luxury       0.97      0.89      0.92       383
      Luxury       0.44      0.76      0.56        46

    accuracy                           0.87       429
   macro avg       0.71      0.82      0.74       429
weighted avg       0.91      0.87      0.89       429



In [32]:
print('Train confusion matrix')
confusion_matrix(y_train, y_train_pred)

Train confusion matrix


array([[2236,  337],
       [  54,  130]], dtype=int64)

In [33]:
print('Test confusion matrix')
confusion_matrix(y_test, y_test_pred)

Test confusion matrix


array([[339,  44],
       [ 11,  35]], dtype=int64)

<font color='red'>Question:</font> Do you see any difference? Why is the difference?

In [ ]:
# The confusion matrix shows more predictions are made on the minor class (luxury category), and while the precision decreas,
# the recall and f1 values increases, so the the model's performance improves.

<font color='red'>Question:</font> Which results are more reasonable? Why?

In [ ]:
# The losistic regression with 'balanced' weight, because it accounts the imbalance of the data, which enable predicting most
# of the minor class events.

<font color='red'>Question:</font> What are the other methods to deal with imbalanced data?

In [ ]:
# 1 use the right metrics, besides precision, recall and f1, also consider using MCC and AUC
# 2 Over-sampling the minor class or under-sampling the major class
# 3 Cluster the major class first
# 4 Use model that is suit for imbalanced data.